In [1]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv()  # load environment variables from .env file
openai.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [3]:
# prepare test files
dfs = []
for file in os.listdir('./clinical_visit_note_summarization_corpus/data/aci-bench/challenge_data/'):
    if 'test' in file and 'metadata' not in file:
        dfs.append(pd.read_csv(f'./clinical_visit_note_summarization_corpus/data/aci-bench/challenge_data/{file}'))
all_dfs = pd.concat(dfs, ignore_index=True)

In [4]:
def model_note_testing(model_name, all_dfs, save_file):
  ai_generated_notes = []
  for i in range(len(all_dfs['dialogue'])):
    completion = client.chat.completions.create(
      model="ft:gpt-3.5-turbo-0125:personal:default-aci-bench:9lDAbkNy",
      messages=[
        {"role": "system", "content": "You are an expert medical professional. Given a clinical dialogue, create a clinical note outlining key dialogue aspects such as 'CHIEF COMPLAINT' (or 'CC'), 'HISTORY OF PRESENT ILLNESS' ('HPI'), 'REVIEW OF SYSTEMS', 'PHYSICAL EXAMINATION', 'VITALS REVIEWED', 'RESULTS', 'ASSESSMENT AND PLAN', 'INSTRUCTIONS', 'CURRENT MEDICATIONS', 'PAST MEDICAL HISTORY', 'EXAM', 'IMPRESSION', 'PLAN', 'ASSESSMENT', 'PAST HISTORY', 'ALLERGIES', 'SOCIAL HISTORY', 'PHYSICAL EXAM', 'PROCEDURE', 'FAMILY HISTORY', 'MEDICATIONS', 'VITALS', 'MEDICAL HISTORY', 'SURGICAL HISTORY'. You will not use all of these aspects in every dialogue, vary it from dialogue to dialogue."},
        {"role": "user", "content": f"Dialogue: {all_dfs['dialogue'][i]}"}
      ]
    )
    ai_generated_notes.append(completion.choices[0].message.content)
  new_dataframe = {'dialogue': all_dfs['dialogue'], 'human_note': all_dfs['note'], 'ai_note':ai_generated_notes}
  new_dataframe = pd.DataFrame(new_dataframe)
  if '.csv' in save_file:
    save_file = save_file.replace('.csv', '')
  new_dataframe.to_csv(f'./testing_files/{save_file}.csv')
  return new_dataframe

In [5]:
df = model_note_testing("ft:gpt-3.5-turbo-0125:personal:default-aci-bench:9lDAbkNy", all_dfs, "default-aci-bench")